In [76]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import io
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import boto3
import sagemaker
from sagemaker import get_execution_role

* Uploading customer & German data to S3 via Sagemaker Jupyter so it can be stored here
* Clean data (drop nan columns, drop non-continuous columns, dropped any columns with zero variance, standardized data using sklearn’s StandardScaler)
* Save cleaned data to S3

In [2]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [3]:
bucket = sagemaker_session.default_bucket()

In [4]:
prefix = 'segmentation'

In [8]:
# azdias_location = sagemaker_session.upload_data('dataset/scaled_mailout_train.csv', key_prefix=prefix)

In [21]:
# data_key= prefix + '/azdias.csv'
data_key= prefix + '/scaled_mailout_train.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

In [22]:
tmp_list = []

for chunk in pd.read_csv(data_location, chunksize=100000):
    tmp_list.append(chunk)
    
df = pd.concat(tmp_list, axis=0)
del tmp_list

In [23]:
df.shape

(42962, 95)

In [24]:
df.head()

,Unnamed: 0,LNR,AGER_TYP,D19_BANKEN_ANZ_12,D19_BANKEN_ANZ_24,D19_BANKEN_DATUM,D19_BANKEN_DIREKT,D19_BANKEN_GROSS,D19_BANKEN_LOKAL,D19_BANKEN_OFFLINE_DATUM,...,SEMIO_REL,SEMIO_SOZ,SEMIO_TRADV,SEMIO_VERT,SHOPPER_TYP,VERS_TYP,ZABEOTYP,RESPONSE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,0,0.020538,0.75,0.000000,0.000000,1.000000,0.0,0.0,0.000000,1.0,...,0.000000,0.666667,0.000000,0.333333,1.00,1.000000,0.4,0.0,1.0,0.375
1,1,0.020631,0.50,0.166667,0.166667,0.444444,0.0,0.0,0.428571,1.0,...,0.333333,0.000000,0.333333,0.500000,0.75,0.666667,0.0,0.0,1.0,0.250
2,2,0.020689,0.50,0.000000,0.166667,0.666667,0.0,0.0,0.000000,1.0,...,0.666667,0.333333,0.000000,1.000000,1.00,0.666667,0.4,0.0,0.0,0.375
3,3,0.017006,0.75,0.000000,0.000000,1.000000,0.0,0.0,0.000000,1.0,...,0.166667,0.666667,0.333333,0.166667,0.50,1.000000,0.4,0.0,1.0,0.375
4,4,0.020771,0.75,0.000000,0.000000,1.000000,0.0,0.0,0.000000,1.0,...,0.500000,0.833333,0.166667,1.000000,0.50,0.666667,0.4,0.0,0.0,0.250


In [13]:
del df